# Data labeling example
This notebook shows how to conduct data labeling and content analysis using [EDSL](https://docs.expectedparrot.com), an open-source library for simulating surveys, experiments and other research with AI agents and large language models. 

Using a dataset of mock customer service tickets as an example, we demonstrate how to: 

1. Import data into EDSL 
2. Create questions about the data 
3. Design an AI agent to answer the questions
4. Select a language model to generate responses
5. Analyze results as a formatted dataset

This workflow can be visualized as follows:
<br><br>
<img src="general_survey.png" width="1000px">
<br><br>

## Technical setup
Before running the code below please ensure that you have completed setup:

* [Install EDSL](https://docs.expectedparrot.com/en/latest/installation.html).
* Create a [Coop account](https://www.expectedparrot.com/login) and activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) OR store your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use.

Our [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) also provides examples of EDSL basic components. 

## Selecting data for review
First we identify some data for review. Data can be created using the EDSL tools or imported from other sources. For purposes of this demo we import a set of hypothetical customer tickets for a transportation app:

In [1]:
tickets = [
    "I just realized I left my phone in the car on my last ride. Can you help me get it back?",
    "I'm unhappy with my recent experience. The driver was very rude and unprofessional.",
    "I was charged more than the estimated fare for my trip yesterday. Can you explain why?",
    "The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.",
    "My driver took a longer route than necessary, resulting in a higher fare. I request a fare adjustment.",
    "I had a great experience with my driver today! Very friendly and efficient service.",
    "I'm concerned about the vehicle's cleanliness. It was not up to the standard I expect.",
    "The app keeps crashing every time I try to book a ride. Please fix this issue.",
    "My driver was exceptional - safe driving, polite, and the car was spotless. Kudos!",
    "I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.",
    "The driver refused to follow my preferred route, which is shorter. I'm not satisfied with the service.",
    "Impressed with the quick response to my ride request and the driver's professionalism.",
    "I was charged for a ride I never took. Please refund me as soon as possible.",
    "The promo code I tried to use didn't work. Can you assist with this?",
    "There was a suspicious smell in the car, and I'm worried about hygiene standards.",
    "My driver was very considerate, especially helping me with my luggage. Appreciate the great service!",
    "The app's GPS seems inaccurate. It directed the driver to the wrong pick-up location.",
    "I want to compliment my driver's excellent navigation and time management during rush hour.",
    "The vehicle didn't match the description in the app. It was confusing and concerning.",
    "I faced an issue with payment processing after my last ride. Can you look into this?",
]

## Constructing questions about the data
Next we create some questions about the data. EDSL provides a variety of question types that we can choose from based on the form of the response that we want to get back from the model (multiple choice, free text, checkbox, linear scale, etc.). [Learn more about question types](https://docs.expectedparrot.com/en/latest/questions.html). 

Note that we use a `{{ placeholder }}` in each question text in order to parameterize the questions with the individual ticket contents in the next step:

In [2]:
from edsl import (
    QuestionMultipleChoice,
    QuestionCheckBox,
    QuestionFreeText,
    QuestionList,
    QuestionYesNo,
    QuestionLinearScale,
)

In [3]:
question_issues = QuestionCheckBox(
    question_name="issues",
    question_text="Check all of the issues mentioned in this ticket: {{ ticket }}",
    question_options=[
        "safety",
        "cleanliness",
        "driver performance",
        "GPS/route",
        "lost item",
        "other",
    ],
)

In [4]:
question_primary_issue = QuestionFreeText(
    question_name="primary_issue",
    question_text="What is the primary issue in this ticket? Ticket: {{ ticket }}",
)

In [5]:
question_accident = QuestionMultipleChoice(
    question_name="accident",
    question_text="If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}",
    question_options=["Yes", "No", "Not applicable"],
)

In [6]:
question_sentiment = QuestionMultipleChoice(
    question_name="sentiment",
    question_text="What is the sentiment of this ticket? Ticket: {{ ticket }}",
    question_options=[
        "Very positive",
        "Somewhat positive",
        "Neutral",
        "Somewhat negative",
        "Very negative",
    ],
)

In [7]:
question_refund = QuestionYesNo(
    question_name="refund",
    question_text="Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}",
)

In [8]:
question_priority = QuestionLinearScale(
    question_name="priority",
    question_text="On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}",
    question_options=[0, 1, 2, 3, 4, 5],
    option_labels={0: "Lowest", 5: "Highest"},
)

## Building a survey
We combine the questions into a survey in order to administer them together:

In [9]:
from edsl import Survey

survey = Survey(
    questions=[
        question_issues,
        question_primary_issue,
        question_accident,
        question_sentiment,
        question_refund,
        question_priority,
    ]
)

Survey questions are administered asynchronously by default. [Learn more about adding conditional logic and memory to your survey](https://docs.expectedparrot.com/en/latest/surveys.html).

We can review our questions in a readable format, or export them as a survey to use with human respondents or at other survey platforms:

In [10]:
# survey

## Designing AI agents
A key feature of EDSL is the ability to create personas for AI agents that the language models are prompted to use in generating responses to the questions. This is done by passing a dictionary of traits to Agent objects:

In [11]:
from edsl import Agent

agent = Agent(
    traits={
        "persona": "You are an expert customer service agent.",
        "years_experience": 15,
    }
)

## Selecting language models
EDSL allows us to select the language models to use in generating results. To see all available models:

In [12]:
from edsl import Model

# Model.available()

Here we select GPT 4 (if no model is specified, GPT 4 previes is used by default):

In [13]:
model = Model("gpt-4")

## Adding data to the questions
We add the contents of each ticket into each question as an independent "scenario" for review. This allows us to create versions of the questions for each job post and deliver them to the model all at once:

In [14]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList(
    Scenario({"ticket": t}) for t in tickets
)

## Running the survey
We run the survey by adding the scenarios, agent and model with the `by()` method and then calling the `run()` method:

In [15]:
results = survey.by(scenarios).by(agent).by(model).run()

This generates a formatted dataset of `Results` that includes information about all the components, including the prompts and responses. We can see a list of all the components:

In [16]:
# results.columns

## Analyzing results
EDSL comes with [built-in methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html). Here we filter, sort, select and print components in a table:

In [17]:
(results
 .filter("priority in ['4', '5']")
 .sort_by("issues", "sentiment")
 .select("ticket", "issues", "primary_issue", "accident", "sentiment", "refund", "priority")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓
┃ scenario         ┃ answer           ┃ answer          ┃ answer         ┃ answer           ┃ answer  ┃ answer    ┃
┃ .ticket          ┃ .issues          ┃ .primary_issue  ┃ .accident      ┃ .sentiment       ┃ .refund ┃ .priority ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩
│ There was a      │ ['cleanliness']  │ The primary     │ No             │ Somewhat         │ No      │ 4         │
│ suspicious smell │                  │ issue in this   │                │ negative         │         │           │
│ in the car, and  │                  │ ticket is a     │                │                  │         │           │
│ I'm worried      │                  │ concern about a │                │                  │         │           │
│ about hygiene    │                  │ suspicious      │                │                  │         │           │
│ standards.       │                  │ smell in the    │                │                  │         │           │
│                  │                  │ car, indicating │                │                  │         │           │
│                  │                  │ possible        │                │                  │         │           │
│                  │                  │ hygiene         │                │                  │         │           │
│                  │                  │ standards       │                │                  │         │           │
│                  │                  │ violation.      │                │                  │         │           │
├──────────────────┼──────────────────┼─────────────────┼────────────────┼──────────────────┼─────────┼───────────┤
│ I'm unhappy with │ ['driver         │ The primary     │ No             │ Very negative    │ No      │ 4         │
│ my recent        │ performance']    │ issue in this   │                │                  │         │           │
│ experience. The  │                  │ ticket is the   │                │                  │         │           │
│ driver was very  │                  │ customer's      │                │                  │         │           │
│ rude and         │                  │ dissatisfaction │                │                  │         │           │
│ unprofessional.  │                  │ with their      │                │                  │         │           │
│                  │                  │ recent          │                │                  │         │           │
│                  │                  │ experience due  │                │                  │         │           │
│                  │                  │ to the driver's │                │                  │         │           │
│                  │                  │ rude and        │                │                  │         │           │
│                  │                  │ unprofessional  │                │                  │         │           │
│                  │                  │ behavior.       │                │                  │         │           │
├──────────────────┼──────────────────┼─────────────────┼────────────────┼──────────────────┼─────────┼───────────┤
│ I just realized  │ ['lost item']    │ The primary     │ Not applicable │ Somewhat         │ No      │ 4         │
│ I left my phone  │                  │ issue in this   │                │ negative         │         │           │
│ in the car on my │                  │ ticket is that  │                │                  │         │           │
│ last ride. Can   │                  │ the customer    │                │                  │         │           │
│ you help me get  │                  │ has left their  │                │                  │         │           │
│ it back?         │                  │ phone in the    

We can apply some lables to our table:

In [18]:
(
    results.select(
        "ticket",
        "issues",
        "primary_issue",
        "accident",
        "sentiment",
        "refund",
        "priority",
    ).print(
        pretty_labels={
            "scenario.ticket": "Ticket",
            "answer.issues": "Issues",
            "answer.primary_issue": "Primary issue",
            "answer.accident": "Accident",
            "answer.sentiment": "Sentiment",
            "answer.refund": "Refund request",
            "answer.priority": "Priority",
        }
    )
)

Ticket,Issues,Primary issue,Accident,Sentiment,Refund request,Priority
I just realized I left my phone in the car on my last ride. Can you help me get it back?,['lost item'],The primary issue in this ticket is that the customer has left their phone in the car during their last ride and needs assistance in retrieving it.,Not applicable,Somewhat negative,No,4
I'm unhappy with my recent experience. The driver was very rude and unprofessional.,['driver performance'],The primary issue in this ticket is the customer's dissatisfaction with their recent experience due to the driver's rude and unprofessional behavior.,No,Very negative,No,4
I was charged more than the estimated fare for my trip yesterday. Can you explain why?,['other'],The primary issue in this ticket is that the customer was charged more than the estimated fare for their trip. They are seeking an explanation for this discrepancy.,Not applicable,Somewhat negative,No,4
"The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.","['safety', 'driver performance']","The primary issue in this ticket is the improper installation of the provided car seat, which led to concerns about child safety. The customer is also requesting driver training to ensure this issue does not recur.",No,Somewhat negative,No,5
"My driver took a longer route than necessary, resulting in a higher fare. I request a fare adjustment.","['driver performance', 'GPS/route']","The primary issue in this ticket is that the customer's driver took a longer route than necessary, which led to a higher fare. The customer is requesting a fare adjustment.",No,Somewhat negative,Yes,3
I had a great experience with my driver today! Very friendly and efficient service.,['driver performance'],There is no issue reported in this ticket. It is a positive feedback about a driver's friendly and efficient service.,Not applicable,Very positive,No,0
I'm concerned about the vehicle's cleanliness. It was not up to the standard I expect.,['cleanliness'],The primary issue in this ticket is the substandard cleanliness of the vehicle.,Not applicable,Somewhat negative,No,3
The app keeps crashing every time I try to book a ride. Please fix this issue.,['other'],The primary issue in this ticket is that the app is crashing every time the customer tries to book a ride.,No,Very negative,No,4
I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.,"['safety', 'driver performance']",The primary issue in this ticket is the customer's feeling of unsafety during their ride due to the driver's erratic behavior.,No,Very negative,No,5
"The driver refused to follow my preferred route, which is shorter. I'm not satisfied with the service.","['driver performance', 'GPS/route']","The primary issue in this ticket is the driver's refusal to follow the customer's preferred route, leading to customer dissatisfaction.",Not applicable,Very negative,No,2


EDSL also comes with methods for accessing results as a dataframe or SQL table:

In [19]:
df = (
    results
    .select(
        "issues",
        "primary_issue",
        "accident",
        "sentiment",
        "refund",
        "priority"
    )
    .to_pandas(remove_prefix=True)
)
df

,issues,primary_issue,accident,sentiment,refund,priority
0,['lost item'],The primary issue in this ticket is that the c...,Not applicable,Somewhat negative,No,4
1,['driver performance'],The primary issue in this ticket is the custom...,No,Very negative,No,4
2,['other'],The primary issue in this ticket is that the c...,Not applicable,Somewhat negative,No,4
3,"['safety', 'driver performance']",The primary issue in this ticket is the improp...,No,Somewhat negative,No,5
4,"['driver performance', 'GPS/route']",The primary issue in this ticket is that the c...,No,Somewhat negative,Yes,3
5,['driver performance'],There is no issue reported in this ticket. It ...,Not applicable,Very positive,No,0
6,['cleanliness'],The primary issue in this ticket is the substa...,Not applicable,Somewhat negative,No,3
7,['other'],The primary issue in this ticket is that the a...,No,Very negative,No,4
8,"['safety', 'driver performance']",The primary issue in this ticket is the custom...,No,Very negative,No,5
9,"['driver performance', 'GPS/route']",The primary issue in this ticket is the driver...,Not applicable,Very negative,No,2


We can also access results as a SQL table:

In [20]:
results.sql("""
select ticket, issues, primary_issue, accident, sentiment, refund, priority 
from self
""", shape="wide")

,ticket,issues,primary_issue,accident,sentiment,refund,priority
0,I just realized I left my phone in the car on ...,['lost item'],The primary issue in this ticket is that the c...,Not applicable,Somewhat negative,No,4
1,I'm unhappy with my recent experience. The dri...,['driver performance'],The primary issue in this ticket is the custom...,No,Very negative,No,4
2,I was charged more than the estimated fare for...,['other'],The primary issue in this ticket is that the c...,Not applicable,Somewhat negative,No,4
3,The car seat provided was not properly install...,"['safety', 'driver performance']",The primary issue in this ticket is the improp...,No,Somewhat negative,No,5
4,"My driver took a longer route than necessary, ...","['driver performance', 'GPS/route']",The primary issue in this ticket is that the c...,No,Somewhat negative,Yes,3
5,I had a great experience with my driver today!...,['driver performance'],There is no issue reported in this ticket. It ...,Not applicable,Very positive,No,0
6,I'm concerned about the vehicle's cleanliness....,['cleanliness'],The primary issue in this ticket is the substa...,Not applicable,Somewhat negative,No,3
7,The app keeps crashing every time I try to boo...,['other'],The primary issue in this ticket is that the a...,No,Very negative,No,4
8,I felt unsafe during my ride due to the driver...,"['safety', 'driver performance']",The primary issue in this ticket is the custom...,No,Very negative,No,5
9,The driver refused to follow my preferred rout...,"['driver performance', 'GPS/route']",The primary issue in this ticket is the driver...,Not applicable,Very negative,No,2


To export results to a CSV file:

In [21]:
results.to_csv("data_labeling_example.csv")

## Posting content to the Coop
We can post any objects to the Coop, including this notebook. Objects can be updated or modified at your Coop account, and shared with others or stored privately (default visibility is *unlisted*):

In [22]:
results.push(description = "Customer service tickets data labeling example", visibility="public")

{'description': 'Customer service tickets data labeling example',
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/8347bb96-951f-4063-a863-a24da3990f52',
 'uuid': '8347bb96-951f-4063-a863-a24da3990f52',
 'version': '0.1.32.dev1',
 'visibility': 'public'}

In [23]:
survey.push(description = "Customer service tickets data labeling example survey", visibility="public")

{'description': 'Customer service tickets data labeling example survey',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/7116a9c0-3bcf-419f-8989-6b1ed5df98c1',
 'uuid': '7116a9c0-3bcf-419f-8989-6b1ed5df98c1',
 'version': '0.1.32.dev1',
 'visibility': 'public'}

In [24]:
from edsl import Notebook

n = Notebook(path="data_labeling_example.ipynb")

n.push(description="Data labeling example", visibility="public") 

{'description': 'Data labeling example',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/830446e3-31e6-4d12-b9ec-ebd38a1ca26e',
 'uuid': '830446e3-31e6-4d12-b9ec-ebd38a1ca26e',
 'version': '0.1.32.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [26]:
n = Notebook(path="data_labeling_example.ipynb")

n.patch(uuid = "830446e3-31e6-4d12-b9ec-ebd38a1ca26e", value = n)

{'status': 'success'}